**References:**  

https://www.datacamp.com/community/tutorials/finance-python-trading

https://github.com/datacamp/datacamp-community-tutorials/blob/master/Python%20Finance%20Tutorial%20For%20Beginners/Python%20For%20Finance%20Beginners%20Tutorial.ipynb

https://pypi.python.org/pypi/fix-yahoo-finance

http://www.learndatasci.com/python-finance-part-yahoo-finance-api-pandas-matplotlib/

In [ ]:
# Import initial libraries

import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

# Imports in order to be able to use Plotly offline.
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import plotly.graph_objs as go

print(__version__) # requires version >= 1.9.0

init_notebook_mode(connected=True)

# Import the Sample worksheet with acquisition dates and initial cost basis:

import pandas as pd
import io
import requests
url="https://raw.githubusercontent.com/duy7590/portfolio_tracker/master/data.csv"
s=requests.get(url).content
portfolio_df = pd.read_csv(io.StringIO(s.decode('utf-8')))
portfolio_df['Acquisition Date'] = portfolio_df['Acquisition Date'].astype('datetime64[ns]') 
portfolio_df['Start of Year'] = portfolio_df['Start of Year'].astype('datetime64[ns]') 
portfolio_df['Cost Basis'] = portfolio_df['Cost Basis'].astype('float64') 
portfolio_df['Unit Cost'] = portfolio_df['Unit Cost'].astype('float64') 
portfolio_df.head(10)


# Confirm that you have 8 values for each column.
portfolio_df.info()



# Date Ranges for OMX25 and for all tickers
# Modify these date ranges each week.
# The below will pull back stock prices from 2010 until end date specified.
start_sp = datetime.datetime(2019, 1, 1)
end_sp = datetime.datetime(2020, 9, 3)

# This variable is used for YTD performance.
end_of_last_year = datetime.datetime(2019, 12, 30)

# These are separate if for some reason want different date range than SP.
stocks_start = datetime.datetime(2019, 1, 1)
stocks_end = datetime.datetime(2020, 9, 3)


# Leveraged from the helpful Datacamp Python Finance trading blog post.

from pandas_datareader import data as pdr
!pip install yfinance
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

OMXH25 = pdr.get_data_yahoo('^OMXH25', start_sp, end_sp)
                          

# Create a dataframe with only the Adj Close column as that's all we need for this analysis.
OMXH25_adj_close = OMXH25[['Adj Close']].reset_index()

# Adj Close for the EOY in 2019 in order to run comparisons versus stocks YTD performances.
OMXH25_adj_close_start = OMXH25_adj_close[OMXH25_adj_close['Date']==end_of_last_year]

# Generate a dynamic list of tickers to pull from Yahoo Finance API based on the imported file with tickers.
tickers = portfolio_df['Ticker'].unique()

# Stock comparison code
def get(tickers, startdate, enddate):
    def data(ticker):
        return (pdr.get_data_yahoo(ticker, start=startdate, end=enddate))
    datas = map(data, tickers)
    return(pd.concat(datas, keys=tickers, names=['Ticker', 'Date']))
               
all_data = get(tickers, stocks_start, stocks_end)

# Also only pulling the ticker, date and adj. close columns for our tickers.
adj_close = all_data[['Adj Close']].reset_index()

# Grabbing the ticker close from the end of last year
adj_close_start = adj_close[adj_close['Date']==end_of_last_year]

# Grab the latest stock close price
adj_close_latest = adj_close[adj_close['Date']==(stocks_end-datetime.timedelta(1))]
adj_close_latest.set_index('Ticker', inplace=True)
portfolio_df.set_index(['Ticker'], inplace=True)

# Merge the portfolio dataframe with the adj close dataframe; they are being joined by their indexes.
merged_portfolio = pd.merge(portfolio_df, adj_close_latest, left_index=True, right_index=True)

# The below creates a new column which is the ticker return; takes the latest adjusted close for each position
# and divides that by the initial share cost.
merged_portfolio['ticker return'] = merged_portfolio['Adj Close'] / merged_portfolio['Unit Cost'] - 1
merged_portfolio.reset_index(inplace=True)

# Here we are merging the new dataframe with the OMXH25 adjusted closes since the sp start price based on 
# each ticker's acquisition date and OMXH25 close date.
merged_portfolio_sp = pd.merge(merged_portfolio, OMXH25_adj_close, left_on='Acquisition Date', right_on='Date')

# We will delete the additional date column which is created from this merge.
# We then rename columns to Latest Date and then reflect Ticker Adj Close and SP 500 Initial Close.

del merged_portfolio_sp['Date_y']

merged_portfolio_sp.rename(columns={'Date_x': 'Latest Date', 'Adj Close_x': 'Ticker Adj Close'
                                    , 'Adj Close_y': 'OMXH25 Initial Close'}, inplace=True)

# This new column determines what OMXH25 equivalent purchase would have been at purchase date of stock.
merged_portfolio_sp['Equiv SP Shares'] = merged_portfolio_sp['Cost Basis'] / merged_portfolio_sp['OMXH25 Initial Close']

# We are joining the developing dataframe with the OMXH25 closes again, this time with the latest close for SP.
merged_portfolio_sp_latest = pd.merge(merged_portfolio_sp, OMXH25_adj_close, left_on='Latest Date', right_on='Date')

# Once again need to delete the new Date column added as it's redundant to Latest Date.  
# Modify Adj Close from the sp dataframe to distinguish it by calling it the SP 500 Latest Close.

del merged_portfolio_sp_latest['Date']
merged_portfolio_sp_latest.rename(columns={'Adj Close': 'OMXH25 Latest Close'}, inplace=True)


# Percent return of SP from acquisition date of position through latest trading day.
merged_portfolio_sp_latest['OMXH25 Return'] = merged_portfolio_sp_latest['OMXH25 Latest Close'] / merged_portfolio_sp_latest['OMXH25 Initial Close'] - 1

# This is a new column which takes the tickers return and subtracts the OMXH25 equivalent range return.
merged_portfolio_sp_latest['Abs. Return Compare'] = merged_portfolio_sp_latest['ticker return'] - merged_portfolio_sp_latest['OMXH25 Return']

# This is a new column where we calculate the ticker's share value by multiplying the original quantity by the latest close.
merged_portfolio_sp_latest['Ticker Share Value'] = merged_portfolio_sp_latest['Quantity'] * merged_portfolio_sp_latest['Ticker Adj Close']

# We calculate the equivalent SP 500 Value if we take the original SP shares * the latest SP 500 share price.
merged_portfolio_sp_latest['OMXH25 Value'] = merged_portfolio_sp_latest['Equiv SP Shares'] * merged_portfolio_sp_latest['OMXH25 Latest Close']

# This is a new column where we take the current market value for the shares and subtract the SP 500 value.
merged_portfolio_sp_latest['Abs Value Compare'] = merged_portfolio_sp_latest['Ticker Share Value'] - merged_portfolio_sp_latest['OMXH25 Value']

# This column calculates profit / loss for stock position.
merged_portfolio_sp_latest['Stock Gain / (Loss)'] = merged_portfolio_sp_latest['Ticker Share Value'] - merged_portfolio_sp_latest['Cost Basis']

# This column calculates profit / loss for OMXH25.
merged_portfolio_sp_latest['OMXH25 Gain / (Loss)'] = merged_portfolio_sp_latest['OMXH25 Value'] - merged_portfolio_sp_latest['Cost Basis']

# Merge the overall dataframe with the adj close start of year dataframe for YTD tracking of tickers.
# Should not need to do the outer join;
merged_portfolio_sp_latest_YTD = pd.merge(merged_portfolio_sp_latest, adj_close_start, on='Ticker')
# , how='outer'

# Deleting date again as it's an unnecessary column.  Explaining that new column is the Ticker Start of Year Close.
del merged_portfolio_sp_latest_YTD['Date']
merged_portfolio_sp_latest_YTD.rename(columns={'Adj Close': 'Ticker Start Year Close'}, inplace=True)

# Join the SP 500 start of year with current dataframe for SP 500 ytd comparisons to tickers.
merged_portfolio_sp_latest_YTD_sp = pd.merge(merged_portfolio_sp_latest_YTD, OMXH25_adj_close_start, left_on='Start of Year', right_on='Date')


# Deleting another unneeded Date column.
del merged_portfolio_sp_latest_YTD_sp['Date']

# Renaming so that it's clear this column is SP 500 start of year close.
merged_portfolio_sp_latest_YTD_sp.rename(columns={'Adj Close': 'OMXH25 Start Year Close'}, inplace=True)

# YTD return for portfolio position.
merged_portfolio_sp_latest_YTD_sp['Share YTD'] = merged_portfolio_sp_latest_YTD_sp['Ticker Adj Close'] / merged_portfolio_sp_latest_YTD_sp['Ticker Start Year Close'] - 1

# YTD return for SP to run compares.
merged_portfolio_sp_latest_YTD_sp['OMXH25 YTD'] = merged_portfolio_sp_latest_YTD_sp['OMXH25 Latest Close'] / merged_portfolio_sp_latest_YTD_sp['OMXH25 Start Year Close'] - 1

merged_portfolio_sp_latest_YTD_sp = merged_portfolio_sp_latest_YTD_sp.sort_values(by='Ticker', ascending=True)

# Cumulative sum of original investment
merged_portfolio_sp_latest_YTD_sp['Cum Invst'] = merged_portfolio_sp_latest_YTD_sp['Cost Basis'].cumsum()

# Cumulative sum of Ticker Share Value (latest FMV based on initial quantity purchased).
merged_portfolio_sp_latest_YTD_sp['Cum Ticker Returns'] = merged_portfolio_sp_latest_YTD_sp['Ticker Share Value'].cumsum()

# Cumulative sum of SP Share Value (latest FMV driven off of initial SP equiv purchase).
merged_portfolio_sp_latest_YTD_sp['Cum OMXH25 Returns'] = merged_portfolio_sp_latest_YTD_sp['OMXH25 Value'].cumsum()

# Cumulative CoC multiple return for stock investments
merged_portfolio_sp_latest_YTD_sp['Cum Ticker ROI Mult'] = merged_portfolio_sp_latest_YTD_sp['Cum Ticker Returns'] / merged_portfolio_sp_latest_YTD_sp['Cum Invst']

4.9.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
Acquisition Date    3 non-null datetime64[ns]
Ticker              3 non-null object
Quantity            3 non-null int64
Unit Cost           3 non-null float64
Cost Basis          3 non-null float64
Start of Year       3 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(1), object(1)
memory usage: 216.0+ bytes
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


## Assessing Where Positions are At versus Highest Close

In [ ]:
# Referencing the adj_close dataframe from above

adj_close.head()

,Ticker,Date,Adj Close
0,NOKIA.HE,2019-01-02,4.762827
1,NOKIA.HE,2019-01-03,4.669588
2,NOKIA.HE,2019-01-04,4.854164
3,NOKIA.HE,2019-01-07,4.958820
4,NOKIA.HE,2019-01-08,5.004490


In [ ]:
portfolio_df.head()

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year
Ticker,,,,,
NOKIA.HE,2020-03-31,1000,2.78,2780.0,2019-12-30
FORTUM.HE,2020-03-31,100,13.40,1340.0,2019-12-30
ETH-EUR,2019-09-16,10,191.98,1919.8,2019-12-30


In [ ]:
# Need to factor in that some positions were purchased much more recently than others.
# Join adj_close dataframe with portfolio in order to have acquisition date.

portfolio_df.reset_index(inplace=True)

adj_close_acq_date = pd.merge(adj_close, portfolio_df, on='Ticker')

adj_close_acq_date.head()

,Ticker,Date,Adj Close,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year
0,NOKIA.HE,2019-01-02,4.762827,2020-03-31,1000,2.78,2780.0,2019-12-30
1,NOKIA.HE,2019-01-03,4.669588,2020-03-31,1000,2.78,2780.0,2019-12-30
2,NOKIA.HE,2019-01-04,4.854164,2020-03-31,1000,2.78,2780.0,2019-12-30
3,NOKIA.HE,2019-01-07,4.958820,2020-03-31,1000,2.78,2780.0,2019-12-30
4,NOKIA.HE,2019-01-08,5.004490,2020-03-31,1000,2.78,2780.0,2019-12-30


In [ ]:
# delete_columns = ['Quantity', 'Unit Cost', 'Cost Basis', 'Start of Year']

del adj_close_acq_date['Quantity']
del adj_close_acq_date['Unit Cost']
del adj_close_acq_date['Cost Basis']
del adj_close_acq_date['Start of Year']

# Sort by these columns in this order in order to make it clearer where compare for each position should begin.
adj_close_acq_date.sort_values(by=['Ticker', 'Acquisition Date', 'Date'], ascending=[True, True, True], inplace=True)

In [ ]:
# Anything less than 0 means that the stock close was prior to acquisition.

adj_close_acq_date['Date Delta'] = pd.to_datetime(adj_close_acq_date['Date']) - pd.to_datetime(adj_close_acq_date['Acquisition Date'])

adj_close_acq_date['Date Delta'] = adj_close_acq_date[['Date Delta']].apply(pd.to_numeric)  

adj_close_acq_date.head()




,Ticker,Date,Adj Close,Acquisition Date,Date Delta
836,ETH-EUR,2019-01-01,122.866203,2019-09-16,-22291200000000000
837,ETH-EUR,2019-01-02,136.962769,2019-09-16,-22204800000000000
838,ETH-EUR,2019-01-03,130.928757,2019-09-16,-22118400000000000
839,ETH-EUR,2019-01-04,135.651688,2019-09-16,-22032000000000000
840,ETH-EUR,2019-01-05,136.578308,2019-09-16,-21945600000000000


In [ ]:
# Modified the dataframe being evaluated to look at highest close which occurred after Acquisition Date (aka, not prior to purchase).

adj_close_acq_date_modified = adj_close_acq_date[adj_close_acq_date['Date Delta']>=0]

adj_close_acq_date_modified.head()

,Ticker,Date,Adj Close,Acquisition Date,Date Delta
1094,ETH-EUR,2019-09-16,179.120483,2019-09-16,0
1095,ETH-EUR,2019-09-17,188.366394,2019-09-16,86400000000000
1096,ETH-EUR,2019-09-18,191.664566,2019-09-16,172800000000000
1097,ETH-EUR,2019-09-19,200.299454,2019-09-16,259200000000000
1098,ETH-EUR,2019-09-20,197.872787,2019-09-16,345600000000000


In [ ]:
# This pivot table will index on the Ticker and Acquisition Date, and find the max adjusted close.

adj_close_pivot = adj_close_acq_date_modified.pivot_table(index=['Ticker', 'Acquisition Date'], values='Adj Close', aggfunc=np.max)

adj_close_pivot.reset_index(inplace=True)

adj_close_pivot

,Ticker,Acquisition Date,Adj Close
0,ETH-EUR,2019-09-16,400.060516
1,FORTUM.HE,2020-03-31,18.360001
2,NOKIA.HE,2020-03-31,4.329500


In [ ]:
# Merge the adj close pivot table with the adj_close table in order to grab the date of the Adj Close High (good to know).

adj_close_pivot_merged = pd.merge(adj_close_pivot, adj_close
                                             , on=['Ticker', 'Adj Close'])

adj_close_pivot_merged.head()

,Ticker,Acquisition Date,Adj Close,Date
0,ETH-EUR,2019-09-16,400.060516,2020-09-01
1,FORTUM.HE,2020-03-31,18.360001,2020-07-23
2,NOKIA.HE,2020-03-31,4.329500,2020-08-04


In [ ]:
# Merge the Adj Close pivot table with the master dataframe to have the closing high since you have owned the stock.

merged_portfolio_sp_latest_YTD_sp_closing_high = pd.merge(merged_portfolio_sp_latest_YTD_sp, adj_close_pivot_merged, on=['Ticker', 'Acquisition Date'])

# Renaming so that it's clear that the new columns are two year closing high and two year closing high date.
merged_portfolio_sp_latest_YTD_sp_closing_high.rename(columns={'Adj Close': 'Closing High Adj Close', 'Date': 'Closing High Adj Close Date'}, inplace=True)

merged_portfolio_sp_latest_YTD_sp_closing_high['Pct off High'] = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker Adj Close'] / merged_portfolio_sp_latest_YTD_sp_closing_high['Closing High Adj Close'] - 1 

merged_portfolio_sp_latest_YTD_sp_closing_high

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,OMXH25 Initial Close,...,OMXH25 Start Year Close,Share YTD,OMXH25 YTD,Cum Invst,Cum Ticker Returns,Cum OMXH25 Returns,Cum Ticker ROI Mult,Closing High Adj Close,Closing High Adj Close Date,Pct off High
0,ETH-EUR,2019-09-16,10,191.98,1919.8,2019-12-30,2020-09-02,371.420593,0.934684,4078.669922,...,4221.97998,2.136814,0.025957,1919.8,3714.205933,2038.838128,1.934684,400.060516,2020-09-01,-0.071589
1,FORTUM.HE,2020-03-31,100,13.40,1340.0,2019-12-30,2020-09-02,17.500000,0.305970,3382.459961,...,4221.97998,-0.067619,0.025957,3259.8,5464.205933,3754.838800,1.676240,18.360001,2020-07-23,-0.046841
2,NOKIA.HE,2020-03-31,1000,2.78,2780.0,2019-12-30,2020-09-02,3.996000,0.437410,3382.459961,...,4221.97998,0.229851,0.025957,6039.8,9460.205984,7314.899894,1.566311,4.329500,2020-08-04,-0.077030


In [ ]:
# Current Share Price versus Closing High Since Purchased

trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Pct off High'][0:10],
    name = 'Pct off High')
    
data = [trace1]

layout = go.Layout(title = 'Adj Close % off of High: Current Share Price versus Closing High Since Purchased'
    , barmode = 'group'
    , yaxis=dict(title='% Below Adj Close High', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.8,y=1)
    )

fig = go.Figure(data=data, layout=layout)
fig.show(renderer="colab")

## YTD and Trailing Stop Charts

In [ ]:
# Imports in order to be able to use Plotly offline.
!pip install plotly==4.9.0
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "colab"


print(__version__) # requires version >= 1.9.0

init_notebook_mode(connected=True)

4.9.0


In [ ]:
# Ploty is an outstanding resource for interactive charts.

trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp['Share YTD'][0:10],
    name = 'Ticker YTD')

trace2 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp['OMXH25 YTD'][0:10],
    name = 'OMXH25 YTD')
    
data = [trace1, trace2]

layout = go.Layout(title = 'YTD Return vs OMXH25 YTD: Yield of tickers (Stock & Crypto) from beginning of this year '
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.8,y=1)
    )

fig = go.Figure(data=data, layout=layout)
fig.show(renderer="colab")

## Total Return Comparison Charts

In [ ]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['ticker return'][0:10],
    name = 'Ticker Total Return')

trace2 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['OMXH25 Return'][0:10],
    name = 'OMXH25 Total Return')
    
data = [trace1, trace2]

layout = go.Layout(title = 'Total Return vs OMXH25: Total return of each tickers since the acquisition date compared with OMX25 return during the same period'
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".2%")
    , xaxis=dict(title='Ticker', tickformat=".2%")
    , legend=dict(x=.8,y=1)
    )

fig = go.Figure(data=data, layout=layout)
fig.show(renderer="colab")

In [ ]:
merged_portfolio_sp_latest_YTD_sp_closing_high.iloc[:,0:20]

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,OMXH25 Initial Close,Equiv SP Shares,OMXH25 Latest Close,OMXH25 Return,Abs. Return Compare,Ticker Share Value,OMXH25 Value,Abs Value Compare,Stock Gain / (Loss),OMXH25 Gain / (Loss),Ticker Start Year Close
0,ETH-EUR,2019-09-16,10,191.98,1919.8,2019-12-30,2020-09-02,371.420593,0.934684,4078.669922,0.470693,4331.569824,0.062005,0.872678,3714.205933,2038.838128,1675.367805,1794.405933,119.038128,118.406952
1,FORTUM.HE,2020-03-31,100,13.40,1340.0,2019-12-30,2020-09-02,17.500000,0.305970,3382.459961,0.396161,4331.569824,0.280598,0.025373,1750.000000,1716.000672,33.999328,410.000000,376.000672,18.769154
2,NOKIA.HE,2020-03-31,1000,2.78,2780.0,2019-12-30,2020-09-02,3.996000,0.437410,3382.459961,0.821887,4331.569824,0.280598,0.156813,3996.000051,3560.061095,435.938957,1216.000051,780.061095,3.249175


## Cumulative Returns Over Time

In [ ]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Stock Gain / (Loss)'][0:10],
    name = 'Ticker Total Return ($)')

trace2 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['OMXH25 Gain / (Loss)'][0:10],
    name = 'OMXH25 Total Return ($)')

trace3 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['ticker return'][0:10],
    name = 'Ticker Total Return %',
    yaxis='y2')

data = [trace1, trace2, trace3]

layout = go.Layout(title = 'Gain / (Loss) Total Return vs OMXH25'
    , barmode = 'group'
    , yaxis=dict(title='Gain / (Loss) (€)')
    , yaxis2=dict(title='Ticker Return', overlaying='y', side='right', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.75,y=1)
    )

fig = go.Figure(data=data, layout=layout)
fig.show(renderer="colab")

In [ ]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Invst'],
    # mode = 'lines+markers',
    name = 'Cum Invst')

trace2 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum OMXH25 Returns'],
    # mode = 'lines+markers',
    name = 'Cum OMXH25 Returns')

trace3 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Ticker Returns'],
    # mode = 'lines+markers',
    name = 'Cum Ticker Returns')

trace4 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Ticker ROI Mult'],
    # mode = 'lines+markers',
    name = 'Cum ROI Mult'
    , yaxis='y2')


data = [trace1, trace2, trace3, trace4]

layout = go.Layout(title = 'Total Cumulative Investments Over Time'
    , barmode = 'group'
    , yaxis=dict(title='Returns')
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.4,y=1)
    , yaxis2=dict(title='Cum ROI Mult', overlaying='y', side='right')               
    )

fig = go.Figure(data=data, layout=layout)
fig.show(renderer="colab")